In [28]:
import pandas as pd

#file_path = r"C:\Users\mwver\Documents\Uni Projects\MENG\Advanced AI\Assignment 2\yelp_labelled"
#file_path = r"C:\Users\mwver\Documents\Uni Projects\MENG\Advanced AI\Assignment 2\amazon_cells_labelled"

#data = pd.read_table("yelp_labelled.txt", names=["Review", "Sentiment"], header=None)
data = pd.read_table("amazon_cells_labelled.txt", names=["Review", "Sentiment"], header=None)
data.head(2)

,Review,Sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1


In [29]:
data["Sentiment"].value_counts()

Sentiment
0    500
1    500
Name: count, dtype: int64

In [30]:
data.dtypes

Review       object
Sentiment     int64
dtype: object

In [31]:
import string
# Get rid of useless characters
transformed_text = []

for review in data["Review"]:

    if not isinstance(review, str):
        review = '' if pd.isna(review) else str(review)

    review = review.translate(str.maketrans('','',string.punctuation))
    review = review.translate(str.maketrans('','',string.digits))
    review = review.lower()
    transformed_text.append(review)

transformed_text[:5]

['so there is no way for me to plug it in here in the us unless i go by a converter',
 'good case excellent value',
 'great for the jawbone',
 'tied to charger for conversations lasting more than  minutesmajor problems',
 'the mic is great']

In [32]:
# Numbered data
transformed_text = pd.Series(transformed_text)
transformed_text[:5]

0    so there is no way for me to plug it in here i...
1                            good case excellent value
2                                great for the jawbone
3    tied to charger for conversations lasting more...
4                                     the mic is great
dtype: object

In [33]:
import spacy
nlp = spacy.load("en_core_web_sm")

# Define custom stop words list with exceptions for negations
custom_stop_words = set(word for word in nlp.Defaults.stop_words if word.lower() not in ['not', 'no', 'never'])

def remove_stop_words(text):
    doc = nlp(text)
    return ' '.join(token.text for token in doc if token.text.lower() not in custom_stop_words)

transformed_text = transformed_text.apply(remove_stop_words)

print(transformed_text[:5])

0                                no way plug converter
1                            good case excellent value
2                                        great jawbone
3    tied charger conversations lasting   minutesma...
4                                            mic great
dtype: object


In [34]:
# lemmatising, bring words to root word
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J":wordnet.ADJ,
                "N":wordnet.NOUN,
                "V":wordnet.VERB,
                "R":wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

# Lemmatize with POS tags
lemmatizer = WordNetLemmatizer()
lemmatized_text = transformed_text.apply(lambda x: ' '.join(lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in x.split()))
lemmatized_text[:5]

0                                no way plug converter
1                            good case excellent value
2                                        great jawbone
3    tie charger conversation last minutesmajor pro...
4                                            mic great
dtype: object

In [35]:
reviews = list(zip(lemmatized_text, data["Sentiment"]))

prepared_data = pd.DataFrame(reviews, columns=["Review", "Sentiment"])
prepared_data["Review"]

0                                  no way plug converter
1                              good case excellent value
2                                          great jawbone
3      tie charger conversation last minutesmajor pro...
4                                              mic great
                             ...                        
995                  screen smudge easily touch ear face
996                           piece junk lose call phone
997                               item not match picture
998                 thing disappoint infra red port irda
999                      not answer call unit never work
Name: Review, Length: 1000, dtype: object

In [36]:
#prepared_data.to_csv('prepared_data.csv', index=False)
prepared_data.to_csv('prepared_data2.csv', index=False)